In [ ]:

from bs4 import BeautifulSoup
import urllib.request
import csv
urlpage = 'https://fr.wikipedia.org/wiki/Liste_des_pays_par_population'

page = urllib.request.urlopen(urlpage)
#on met dans soup le contenu html
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table')
print(type(table))
results = table.find_all('tr')
print('Number of results', len(results))
fichier = open("pays.txt", "w")    
#dans l'article, le nom est dans une balise H3

for result in results :
    for i in range(2,len(results)):
          info = results[i].find_all('td')
          for infos in info:
                nom = infos.getText()
                fichier.write(nom + ";") 

In [ ]:

from bs4 import BeautifulSoup
import pandas as pd
import requests



#Extraction des données pays + population de wikipédia
url = 'https://fr.wikipedia.org/wiki/Liste_des_pays_par_population'
response = requests.get(url)
soup = BeautifulSoup(response.text)
data = soup.find_all('table')[0]
df = pd.read_html(str(data))[0]
del df['Population au 1er juillet 2021']
del df['Rang (2021)']
df.head()


In [ ]:

#importation de ascii_letters pour nettoyer le dataframe
import string
string.ascii_letters
'abcdefghijklmnopqrstuvwxyz'
alpha=list(string.ascii_lowercase)
v=list(string.ascii_letters)
print([v[1]])

In [ ]:

#Création d'une liste ou les crochets seront supprimées
liste=[]
liste1=[]
for x in df['Pays ou territoire'] :
    for i in x:
        for j in range(len(v)):
                x= x.replace("[" +v[j] + "]"," ")
    liste.append(x)
#print(liste)


In [ ]:

#Remplaçement des valeurs de la colonne pays du dataframe par celles de la liste
for i in range (len(liste)):
    y=df['Pays ou territoire']==liste
    if y[i]== False:
        df['Pays ou territoire'][i]= liste[i]


In [ ]:

df.head()


In [ ]:

#écriture de notre dataframe dans un fichier texte
df.to_csv(r"pays.txt",sep= ";" ,index = None, header = False)



In [ ]:

#Extraction des données pays + pib de wikipédia
urlpage = 'https://fr.wikipedia.org/wiki/Liste_des_pays_par_PIB_nominal'
response = requests.get(urlpage)
soup = BeautifulSoup(response.text)
data = soup.find_all('table')[0]
df2 = pd.read_html(str(data))[0]
df2.head()

In [ ]:

#importation de ascii_letters et création d'une liste pour nettoyer le dataframe
import string
string.ascii_letters
'abcdefghijklmnopqrstuvwxyz'
alpha=list(string.ascii_lowercase)
v=list(string.ascii_letters)
p=[1,2,3,4,5,6,7,8,9,10]

In [ ]:
#Création d'une liste ou les crochets seront supprimées
liste=[]
liste1=[]
for x in df2['Pays ou territoire'] :
    for i in x:
        for j in range(len(v)):
            for r in range(len(p)):
                x= x.replace("[" +v[j] + "]"," ")
    liste.append(x)
for x in liste:
    for i in x:
        for j in range(len(p)):
             x= x.replace("["+ str(p[j])+"]"," ")
    liste1.append(x)
#print(liste1)

In [ ]:

#Remplaçement des valeurs de la colonne pays du dataframe par celles de la liste
for i in range (len(liste1)):
    y=df2['Pays ou territoire']==liste1
    if y[i]== False:
        df2['Pays ou territoire'][i]= liste1[i]
        

In [ ]:

df2.head()

In [ ]:

#écriture de notre dataframe dans un fichier texte
df2.to_csv(r"pib.txt",sep= ";" ,index = None, header = False)



In [ ]:

import sqlite3
import io
#ouverture des fichiers texte
fich = open("pays.txt", "r")
fi= open("pib.txt", "r")
lignes = fich.readlines()
lines = fi.readlines()
#connexion à la base de données
connec = sqlite3.connect('jean.sqlite3')
curseu = connec.cursor()
#Création des tables
#curseu.execute("create TABLE pay( pays str,popu int)")
#curseu.execute("CREATE TABLE pib( rang int,pays str, pib int)")
#Ecriture des données dans les tables
for ligne in lignes:
    element = ligne.split(";")
    curseu.execute("INSERT INTO pay VALUES(?,?)", element)
connec.commit()
for line in lines:
    elements = line.split(";")
    curseu.execute("INSERT INTO pib VALUES(?,?,?)", elements)
connec.commit()


In [ ]:

import sqlite3
#Connexion à la base de données
connec = sqlite3.connect('jean.sqlite3')
curseu = connec.cursor()
#Jointure des deux tables
curseu.execute("SELECT pay.pays, pay.popu , pib.pib FROM pay JOIN pib ON pay.pays = pib.pays ")
fichier = open("exo.txt", "w", encoding= "UTF-8")
#Ecriture du résultat de la jointure dans un fichier texte
valeurs = curseu.fetchall()
for v in valeurs:
     fichier.write(f"{v}\n")
connec.close()


In [ ]:

import unidecode
fich = open("web.txt", "w")
fichier = open("exo.txt", "r")
lines = fichier.readlines()
for line in lines:
    line = str.replace(line,r"\xa0" , '' )
    fich.write(line)
    print(line)


In [ ]:

fich = open("webscraping.txt", "w")
fichier = open("web.txt", "r")
lines = fichier.readlines()
for line in lines:
    line = str.replace(line,r"\n" , '' )
    fich.write(line)
    print(line)

In [ ]:

import openpyxl
fichier = open("exo.txt", "r")
#la variable “classeur”
classeur = openpyxl.Workbook()
 
#sauvegarde d’un fichier 
classeur.save('test.xlsx')
 
#Ouverture d’un fichier
classeur =openpyxl.load_workbook('test.xlsx')

feuille = classeur[classeur.sheetnames[0]]
lignes = fichier.readlines()
for i in range(1,len(lignes)):
        feuille.cell(i,1).value = lignes[i]
classeur.save('test.xlsx')
classeur.close